# 0. DCGAN

### Reference
- https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/dcgan/dcgan.py
- paper : https://arxiv.org/abs/1511.06434

# 1. Library Import

In [1]:
!pip install easydict

  Created wheel for easydict: filename=easydict-1.9-py3-none-any.whl size=6350 sha256=468b2bd92eb72ba0c89df5ba93becf0c665b058e9c66e52dd3842ae4566415ab
  Stored in directory: /root/.cache/pip/wheels/88/96/68/c2be18e7406804be2e593e1c37845f2dd20ac2ce1381ce40b0
Successfully built easydict


In [2]:
# import argparse
import easydict
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch


In [3]:
# make images folder
os.makedirs('images', exist_ok = True)

# 2. Parameter Setting

In [4]:
opt = easydict.EasyDict({"n_epochs" : 10, "batch_size" : 64,
                         "lr" : 0.0002, "b1" : 0.5, "b2" : 0.999,
                         "n_cpu" : 8, "latent_dim" : 100,
                         "img_size" : 32, "channels" : 1,
                         "sample_interval" : 400})

In [5]:
print(opt)

{'n_epochs': 10, 'batch_size': 64, 'lr': 0.0002, 'b1': 0.5, 'b2': 0.999, 'n_cpu': 8, 'latent_dim': 100, 'img_size': 32, 'channels': 1, 'sample_interval': 400}


In [6]:
cuda = True if torch.cuda.is_available() else False

In [7]:
print(cuda)

True


# 3. Model

#### 3.1 Weights Init

In [8]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [20]:
gg = Generator()

In [21]:
gg.apply(weights_init_normal)

Generator(
  (l1): Sequential(
    (0): Linear(in_features=100, out_features=8192, bias=True)
  )
  (conv_blocks): Sequential(
    (0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Upsample(scale_factor=2.0, mode=nearest)
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Upsample(scale_factor=2.0, mode=nearest)
    (6): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Conv2d(64, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): Tanh()
  )
)

In [22]:
gg

Generator(
  (l1): Sequential(
    (0): Linear(in_features=100, out_features=8192, bias=True)
  )
  (conv_blocks): Sequential(
    (0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Upsample(scale_factor=2.0, mode=nearest)
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Upsample(scale_factor=2.0, mode=nearest)
    (6): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Conv2d(64, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): Tanh()
  )
)

#### 3.2 Generator Model

In [9]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        # 32 // 4 = 8
        # 128 * 8 ** 2 = 8192
        self.init_size = opt.img_size // 4
        self.l1 = nn.Sequential(nn.Linear(opt.latent_dim, 128 * self.init_size ** 2))
        
        self.conv_blocks = nn.Sequential(nn.BatchNorm2d(128),
                                         nn.Upsample(scale_factor = 2),
                                         nn.Conv2d(128, 128, 3, stride = 1, padding = 1),
                                         nn.BatchNorm2d(128, 0.8),
                                         nn.LeakyReLU(0.2, inplace = True),
                                         nn.Upsample(scale_factor = 2),
                                         nn.Conv2d(128, 64, 3, stride = 1, padding = 1),
                                         nn.BatchNorm2d(64, 0.8),
                                         nn.LeakyReLU(0.2, inplace = True),
                                         nn.Conv2d(64, opt.channels, 3, stride = 1, padding = 1),
                                         nn.Tanh()
                                        )
        
    def forward(self, z):
        out = self.l1(z)
        # batch size * 128 channel * 8 * 8
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

#### 3.3 Discriminator

In [10]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        def discriminator_block(in_filters, out_filters, bn = True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1),
                     nn.LeakyReLU(0.2, inplace = True),
                     nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block
        
        self.model = nn.Sequential(*discriminator_block(opt.channels, 16, bn = False),
                                   *discriminator_block(16, 32),
                                   *discriminator_block(32, 64),
                                   *discriminator_block(64,128)
                                  )
        
        # The height and width of downsampled image
        ds_size = opt.img_size // 2 ** 4
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1),
                                       nn.Sigmoid()
                                      )
        
    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)
        return validity

#### 3.4 Loss Function

In [11]:
adversarial_loss = torch.nn.BCELoss()

# 4. Data Loader and Model Training

In [12]:
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

In [13]:
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

In [14]:
# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

Discriminator(
  (model): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout2d(p=0.25, inplace=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Dropout2d(p=0.25, inplace=False)
    (6): BatchNorm2d(32, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Dropout2d(p=0.25, inplace=False)
    (10): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (12): LeakyReLU(negative_slope=0.2, inplace=True)
    (13): Dropout2d(p=0.25, inplace=False)
    (14): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
  )
  (adv_laye

In [15]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

--2021-04-24 12:07:34--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2021-04-24 12:07:34--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz’

MNIST.tar.gz            [           <=>      ]  33.20M  15.0MB/s    in 2.2s    

2021-04-24 12:07:36 (15.0 MB/s) - ‘MNIST.tar.gz’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw/tra

In [16]:
# Condigure data loader
os.makedirs("./", exist_ok = True)
dataloader = torch.utils.data.DataLoader(
                datasets.MNIST(
                    "./",
                    train = True,
                    download = True,
                    transform = transforms.Compose(
                        [transforms.Resize(opt.img_size), transforms.ToTensor(),
                         transforms.Normalize([0.5], [0.5])]
                    )
                ),
                batch_size = opt.batch_size,
                shuffle = True
            )

In [17]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr = opt.lr, betas = (opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr = opt.lr, betas = (opt.b1, opt.b2))

In [18]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [19]:
# Training
for epoch in range(opt.n_epochs):
    for i, (imgs, _) in enumerate(dataloader):
        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad = False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad = False)
        
        # Configure input
        real_imgs = Variable(imgs.type(Tensor))
        
        ###################
        # Train Generator #
        ###################
        
        optimizer_G.zero_grad()
        
        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))
        
        # Generator a batch of images
        gen_imgs = generator(z)
        
        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)
        
        g_loss.backward()
        optimizer_G.step()
        
        
        #######################
        # Train Discriminator #
        #######################
        
        optimizer_D.zero_grad()
        
        # Measure discriminator's aility to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2
        
        d_loss.backward()
        optimizer_D.step()
        
        if i % 20 == 0:
            print("[Epoch %d / %d] [Batch %d / %d] [D loss: %f] [G loss: %f]"
                  % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
                 )
        
        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow = 5, normalize = True)

[Epoch 0 / 10] [Batch 0 / 938] [D loss: 0.693363] [G loss: 0.672970]
[Epoch 0 / 10] [Batch 20 / 938] [D loss: 0.691747] [G loss: 0.675183]
[Epoch 0 / 10] [Batch 40 / 938] [D loss: 0.683623] [G loss: 0.688669]
[Epoch 0 / 10] [Batch 60 / 938] [D loss: 0.686531] [G loss: 0.671084]
[Epoch 0 / 10] [Batch 80 / 938] [D loss: 0.689837] [G loss: 0.671968]
[Epoch 0 / 10] [Batch 100 / 938] [D loss: 0.693792] [G loss: 0.705220]
[Epoch 0 / 10] [Batch 120 / 938] [D loss: 0.696985] [G loss: 0.702821]
[Epoch 0 / 10] [Batch 140 / 938] [D loss: 0.694378] [G loss: 0.684572]
[Epoch 0 / 10] [Batch 160 / 938] [D loss: 0.690753] [G loss: 0.727053]
[Epoch 0 / 10] [Batch 180 / 938] [D loss: 0.686377] [G loss: 0.691130]
[Epoch 0 / 10] [Batch 200 / 938] [D loss: 0.692442] [G loss: 0.713231]
[Epoch 0 / 10] [Batch 220 / 938] [D loss: 0.695171] [G loss: 0.696384]
[Epoch 0 / 10] [Batch 240 / 938] [D loss: 0.694592] [G loss: 0.687491]
[Epoch 0 / 10] [Batch 260 / 938] [D loss: 0.696945] [G loss: 0.704127]
[Epoch 0 / 1